In [9]:
from __future__ import annotations
import ast
import re
from dataclasses import dataclass
from typing import List, Dict, Tuple
import pandas as pd
import numpy as np
import os

In [72]:
df = pd.read_csv("/home/provido/provido/luna/image_data/scin_meta/scin_assigned_target_labels.csv")


In [ ]:
cases = pd.read_csv("/home/provido/provido/luna/image_data/scin_meta/scin_cases.csv")
cases

In [16]:
# Combine all three shot type columns and get value counts
shot_type_counts = pd.concat([
    cases['image_1_shot_type'],
    cases['image_2_shot_type'],
    cases['image_3_shot_type']
]).value_counts(dropna=False)

print("Combined shot type value counts:")
print(shot_type_counts)
print(f"\nTotal images: {shot_type_counts.sum()}")

Combined shot type value counts:
CLOSE_UP       5029
NaN            4691
AT_AN_ANGLE    3015
AT_DISTANCE    2364
Name: count, dtype: int64

Total images: 15099


In [22]:
# Keep rows that have "CLOSE_UP" in any of the three shot type columns
cases_closeup = cases[
    (cases['image_1_shot_type'] == 'CLOSE_UP') |
    (cases['image_2_shot_type'] == 'CLOSE_UP') |
    (cases['image_3_shot_type'] == 'CLOSE_UP')
]

print(f"Original cases: {len(cases)}")
print(f"Cases with CLOSE_UP: {len(cases_closeup)}")
print(f"Cases dropped: {len(cases) - len(cases_closeup)}")

# Update the dataframe
cases = cases_closeup
cases


Original cases: 5029
Cases with CLOSE_UP: 5029
Cases dropped: 0


,case_id,source,release,year,age_group,sex_at_birth,fitzpatrick_skin_type,race_ethnicity_american_indian_or_alaska_native,race_ethnicity_asian,race_ethnicity_black_or_african_american,...,related_category,condition_duration,image_1_path,image_2_path,image_3_path,image_1_shot_type,image_2_shot_type,image_3_shot_type,combined_race,race_ethnicity_two_or_more_after_mitigation
0,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NaN,NaN,NaN,NaN,...,RASH,ONE_DAY,dataset/images/-3205742176803893704.png,NaN,NaN,CLOSE_UP,NaN,NaN,NaN,NaN
1,-1002039107727665188,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NaN,NaN,NaN,NaN,...,NaN,NaN,dataset/images/-4762289084741430925.png,NaN,NaN,CLOSE_UP,NaN,NaN,NaN,NaN
2,-1003358831658393077,SCIN,1.0.0,2023,AGE_18_TO_29,MALE,NONE_IDENTIFIED,NaN,NaN,NaN,...,OTHER_ISSUE_DESCRIPTION,ONE_TO_FOUR_WEEKS,dataset/images/-4027806997035329030.png,NaN,NaN,CLOSE_UP,NaN,NaN,HISPANIC_LATINO_OR_SPANISH_ORIGIN,NaN
3,-1003826561155964328,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NaN,NaN,NaN,NaN,...,OTHER_ISSUE_DESCRIPTION,ONE_DAY,dataset/images/-5332065579713135540.png,dataset/images/-6353431708064969797.png,dataset/images/742075435141960831.png,AT_DISTANCE,AT_AN_ANGLE,CLOSE_UP,NaN,NaN
4,-1003844406100696311,SCIN,1.0.0,2023,AGE_40_TO_49,FEMALE,FST3,NaN,NaN,NaN,...,RASH,ONE_DAY,dataset/images/-3799298995660217860.png,dataset/images/-5881426422999442186.png,dataset/images/5854025080806696361.png,AT_AN_ANGLE,AT_DISTANCE,CLOSE_UP,WHITE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5028,973759811984683137,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,FST3,NaN,NaN,NaN,...,RASH,LESS_THAN_ONE_WEEK,dataset/images/32575980331712012.png,dataset/images/5167477372002548597.png,NaN,CLOSE_UP,AT_AN_ANGLE,NaN,TWO_OR_MORE_AFTER_MITIGATION,YES
5029,973872028150293734,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NaN,NaN,NaN,NaN,...,RASH,LESS_THAN_ONE_WEEK,dataset/images/-5315065439551573643.png,dataset/images/-8082245132295705823.png,dataset/images/1606190140951179537.png,CLOSE_UP,AT_DISTANCE,AT_AN_ANGLE,NaN,NaN
5030,983265958160210646,SCIN,1.0.0,2023,AGE_18_TO_29,FEMALE,FST5,NaN,NaN,YES,...,RASH,LESS_THAN_ONE_WEEK,dataset/images/-4723634841049886674.png,dataset/images/6753903069158024633.png,dataset/images/6948340611762812997.png,CLOSE_UP,AT_AN_ANGLE,AT_DISTANCE,BLACK_OR_AFRICAN_AMERICAN,NaN
5031,995820220956352730,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,FST5,NaN,NaN,NaN,...,RASH,ONE_TO_FOUR_WEEKS,dataset/images/-3758258982362095839.png,dataset/images/-8850087380548289976.png,dataset/images/1619983824061709467.png,AT_DISTANCE,CLOSE_UP,AT_AN_ANGLE,TWO_OR_MORE_AFTER_MITIGATION,YES


In [ ]:
# df = df[df['assigned_target_label'] != "unclear"].reset_index(drop=True)
# df

,case_id,dermatologist_gradable_for_skin_condition_1,dermatologist_gradable_for_skin_condition_2,dermatologist_gradable_for_skin_condition_3,dermatologist_skin_condition_on_label_name,dermatologist_skin_condition_confidence,weighted_skin_condition_label,dermatologist_gradable_for_fitzpatrick_skin_type_1,dermatologist_gradable_for_fitzpatrick_skin_type_2,dermatologist_gradable_for_fitzpatrick_skin_type_3,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,gradable_for_monk_skin_tone_india,gradable_for_monk_skin_tone_us,monk_skin_tone_label_india,monk_skin_tone_label_us,conditions_list,assigned_target_label,assigned_target_scores
0,-1002039107727665188,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,NaN,NaN,"['Prurigo nodularis', 'Actinic Keratosis', 'SC...","[4, 3, 4]","{'Prurigo nodularis': 0.41, 'SCC/SCCIS': 0.41,...",YES,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0,"['prurigo nodularis', 'actinic keratosis', 'sc...",scleroderma,"{'melanoma': 0.3360576927661896, 'psoriasis': ..."
1,-1003358831658393077,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,NaN,NaN,"['Impetigo', 'Herpes Zoster', 'Bullous dermati...","[2, 1, 1]","{'Impetigo': 0.55, 'Herpes Zoster': 0.23, 'Bul...",YES,NaN,NaN,FST4,NaN,NaN,True,True,3.0,4.0,"['impetigo', 'herpes zoster', 'bullous dermati...",dermatitis,"{'melanoma': 0.3302953541278839, 'psoriasis': ..."
2,-1013831220015814987,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,NaN,NaN,"['Inflicted skin lesions', 'Basal Cell Carcino...","[2, 4, 1]","{'Basal Cell Carcinoma': 0.55, 'Inflicted skin...",YES,NaN,NaN,FST3,NaN,NaN,True,True,3.0,4.0,"['inflicted skin lesions', 'basal cell carcino...",melanoma,"{'melanoma': 0.5524225234985352, 'psoriasis': ..."
3,-101827005996397499,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,"['Psoriasis', 'Eczema', 'Eczema', 'Acute and c...","[1, 3, 3, 2, 4, 3]","{'Eczema': 0.67, 'Psoriasis': 0.11, 'Acute and...",YES,YES,YES,FST3,FST4,FST4,True,True,2.0,4.0,"['psoriasis', 'eczema', 'eczema', 'acute and c...",psoriasis,"{'melanoma': 0.0, 'psoriasis': 1.0, 'systemic ..."
4,-1033431645373513522,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,NaN,NaN,['Urticaria'],[5],{'Urticaria': 1.0},YES,NaN,NaN,FST3,NaN,NaN,True,True,3.0,3.0,['urticaria'],dermatitis,"{'melanoma': 0.232705220580101, 'psoriasis': 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064,9204674379234810196,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,NaN,NaN,"['Urticaria', 'Cutaneous lupus']","[5, 1]","{'Urticaria': 0.67, 'Cutaneous lupus': 0.33}",NO,NaN,NaN,NaN,NaN,NaN,True,True,3.0,4.0,"['urticaria', 'cutaneous lupus']",systemic lupus erythematosus,"{'melanoma': 0.0, 'psoriasis': 0.0, 'systemic ..."
1065,9212720578959463880,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,NaN,NaN,"['Acne', 'Rosacea', 'Hemangioma']","[3, 4, 5]","{'Hemangioma': 0.55, 'Rosacea': 0.27, 'Acne': ...",YES,NaN,NaN,FST2,NaN,NaN,True,True,2.0,2.0,"['acne', 'rosacea', 'hemangioma']",dermatitis,"{'melanoma': 0.36434677243232727, 'psoriasis':..."
1066,935346003160692168,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,NaN,NaN,['Psoriasis'],[5],{'Psoriasis': 1.0},YES,NaN,NaN,FST3,NaN,NaN,True,True,3.0,5.0,['psoriasis'],psoriasis,"{'melanoma': 0.0, 'psoriasis': 1.0, 'systemic ..."
1067,973759811984683137,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,"['CD - Contact dermatitis', 'Allergic Contact ...","[5, 4, 5]","{'CD - Contact dermatitis': 0.33, 'Allergic Co...",YES,YES,YES,FST2,FST2,FST3,True,True,4.0,4.0,"['cd - contact dermatitis', 'allergic contact ...",psoriasis,"{'melanoma': 0.23959782719612122, 'psoriasis':..."


### Pull the paths for the image close up and assign to case id.
##### Later we will match case id 

In [23]:
def get_closeup_path(row):
    """
    Extract the image path that corresponds to CLOSE_UP shot type.
    Returns the path and performs validation.
    """
    # Check each image position
    if row['image_1_shot_type'] == 'CLOSE_UP':
        return row['image_1_path']
    elif row['image_2_shot_type'] == 'CLOSE_UP':
        return row['image_2_path']
    elif row['image_3_shot_type'] == 'CLOSE_UP':
        return row['image_3_path']
    else:
        return None  # Should not happen given our filter, but safety check

# Apply the function to get closeup paths
cases['closeup_path'] = cases.apply(get_closeup_path, axis=1)

# VALIDATION: Ensure we got a path for every row
print("Validation checks:")
print(f"Total cases: {len(cases)}")
print(f"Cases with closeup_path: {cases['closeup_path'].notna().sum()}")
print(f"Cases WITHOUT closeup_path: {cases['closeup_path'].isna().sum()}")

# Additional validation: verify the path corresponds to CLOSE_UP
def validate_closeup_match(row):
    """Double-check that the extracted path matches a CLOSE_UP shot type"""
    path = row['closeup_path']
    if pd.isna(path):
        return False
    
    # Check if this path matches any CLOSE_UP position
    if row['image_1_path'] == path and row['image_1_shot_type'] == 'CLOSE_UP':
        return True
    if row['image_2_path'] == path and row['image_2_shot_type'] == 'CLOSE_UP':
        return True
    if row['image_3_path'] == path and row['image_3_shot_type'] == 'CLOSE_UP':
        return True
    return False

cases['validation_passed'] = cases.apply(validate_closeup_match, axis=1)
print(f"\nValidation passed: {cases['validation_passed'].sum()} / {len(cases)}")

if not cases['validation_passed'].all():
    print("WARNING: Some paths did not pass validation!")
    print(cases[~cases['validation_passed']][['case_id', 'image_1_shot_type', 'image_2_shot_type', 'image_3_shot_type']])

# Create the dictionary: case_id -> closeup_path
case_id_to_closeup_path = dict(zip(cases['case_id'], cases['closeup_path']))

print(f"\nDictionary created with {len(case_id_to_closeup_path)} entries")
print("\nSample entries:")
for case_id, path in list(case_id_to_closeup_path.items())[:5]:
    print(f"  {case_id}: {path}")

Validation checks:
Total cases: 5029
Cases with closeup_path: 5029
Cases WITHOUT closeup_path: 0

Validation passed: 5029 / 5029

Dictionary created with 5029 entries

Sample entries:
  -1000600354148496558: dataset/images/-3205742176803893704.png
  -1002039107727665188: dataset/images/-4762289084741430925.png
  -1003358831658393077: dataset/images/-4027806997035329030.png
  -1003826561155964328: dataset/images/742075435141960831.png
  -1003844406100696311: dataset/images/5854025080806696361.png


In [26]:
# Replace 'dataset/images' with the actual path
case_id_to_closeup_path = {
    case_id: path.replace('dataset/images', '/home/provido/provido/luna/image_data/scin')
    for case_id, path in case_id_to_closeup_path.items()
}

print("Updated dictionary with full paths:")
print("\nSample entries:")
for case_id, path in list(case_id_to_closeup_path.items())[:5]:
    print(f"  {case_id}: {path}")


Updated dictionary with full paths:

Sample entries:
  -1000600354148496558: /home/provido/provido/luna/image_data/scin/-3205742176803893704.png
  -1002039107727665188: /home/provido/provido/luna/image_data/scin/-4762289084741430925.png
  -1003358831658393077: /home/provido/provido/luna/image_data/scin/-4027806997035329030.png
  -1003826561155964328: /home/provido/provido/luna/image_data/scin/742075435141960831.png
  -1003844406100696311: /home/provido/provido/luna/image_data/scin/5854025080806696361.png


In [74]:
# Validation: Check if ALL files actually exist
print("Validating file existence for ALL files:")
import os

existing_files = {}
missing_files = {}

for case_id, path in case_id_to_closeup_path.items():
    if os.path.exists(path):
        existing_files[case_id] = path
    else:
        missing_files[case_id] = path
        #missing_files.append((case_id, path))

print(f"\nTotal files: {len(case_id_to_closeup_path)}")
print(f"Files that exist: {len(existing_files)}")
print(f"Files NOT found: {len(missing_files)}")
print(f"Success rate: {len(existing_files)/len(case_id_to_closeup_path)*100:.2f}%")

missing_ids = list(missing_files.keys())

Validating file existence for ALL files:

Total files: 5029
Files that exist: 4925
Files NOT found: 104
Success rate: 97.93%


In [75]:
#We have 104 missing files due to download failures. Fortunately, most of these are from our higher distribution classes (dermatitis/psoriasis)
df2 = df[~df['case_id'].isin(missing_ids)]
df2

,case_id,dermatologist_gradable_for_skin_condition_1,dermatologist_gradable_for_skin_condition_2,dermatologist_gradable_for_skin_condition_3,dermatologist_skin_condition_on_label_name,dermatologist_skin_condition_confidence,weighted_skin_condition_label,dermatologist_gradable_for_fitzpatrick_skin_type_1,dermatologist_gradable_for_fitzpatrick_skin_type_2,dermatologist_gradable_for_fitzpatrick_skin_type_3,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,gradable_for_monk_skin_tone_india,gradable_for_monk_skin_tone_us,monk_skin_tone_label_india,monk_skin_tone_label_us,conditions_list,assigned_target_label,assigned_target_scores
0,-1000600354148496558,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,NaN,NaN,"['Inflicted skin lesions', 'Eczema', 'Irritant...","[4, 4, 3]","{'Inflicted skin lesions': 0.41, 'Eczema': 0.4...",YES,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0,"['inflicted skin lesions', 'eczema', 'irritant...",unclear,"{'melanoma': 0.41149958968162537, 'psoriasis':..."
1,-1002039107727665188,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,NaN,NaN,"['Prurigo nodularis', 'Actinic Keratosis', 'SC...","[4, 3, 4]","{'Prurigo nodularis': 0.41, 'SCC/SCCIS': 0.41,...",YES,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0,"['prurigo nodularis', 'actinic keratosis', 'sc...",scleroderma,"{'melanoma': 0.3360576927661896, 'psoriasis': ..."
2,-1003358831658393077,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,NaN,NaN,"['Impetigo', 'Herpes Zoster', 'Bullous dermati...","[2, 1, 1]","{'Impetigo': 0.55, 'Herpes Zoster': 0.23, 'Bul...",YES,NaN,NaN,FST4,NaN,NaN,True,True,3.0,4.0,"['impetigo', 'herpes zoster', 'bullous dermati...",dermatitis,"{'melanoma': 0.3302953541278839, 'psoriasis': ..."
3,-1003826561155964328,NO_IMAGE_QUALITY_INSUFFICIENT,NaN,NaN,[],[],{},NO,NaN,NaN,NaN,NaN,NaN,True,True,2.0,4.0,[],unclear,"{'melanoma': 0.0, 'psoriasis': 0.0, 'systemic ..."
4,-1003844406100696311,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,NaN,NaN,"['Lichen planus/lichenoid eruption', 'Follicul...","[1, 1, 1]","{'Lichen planus/lichenoid eruption': 0.33, 'Fo...",YES,NaN,NaN,FST1,NaN,NaN,True,True,1.0,1.0,"['lichen planus/lichenoid eruption', 'follicul...",unclear,"{'melanoma': 0.33381786942481995, 'psoriasis':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5028,973759811984683137,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,"['CD - Contact dermatitis', 'Allergic Contact ...","[5, 4, 5]","{'CD - Contact dermatitis': 0.33, 'Allergic Co...",YES,YES,YES,FST2,FST2,FST3,True,True,4.0,4.0,"['cd - contact dermatitis', 'allergic contact ...",psoriasis,"{'melanoma': 0.23959782719612122, 'psoriasis':..."
5029,973872028150293734,NO_IMAGE_QUALITY_INSUFFICIENT,NaN,NaN,[],[],{},NO,NaN,NaN,NaN,NaN,NaN,True,True,2.0,1.0,[],unclear,"{'melanoma': 0.0, 'psoriasis': 0.0, 'systemic ..."
5030,983265958160210646,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,NaN,NaN,"['Impetigo', 'Foreign body']","[1, 1]","{'Impetigo': 0.5, 'Foreign body': 0.5}",YES,NaN,NaN,FST5,NaN,NaN,True,True,4.0,5.0,"['impetigo', 'foreign body']",unclear,"{'melanoma': 0.2440209686756134, 'psoriasis': ..."
5031,995820220956352730,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,NaN,NaN,"['Erythema gyratum repens', 'Seborrheic Dermat...","[2, 2, 2]","{'Erythema gyratum repens': 0.33, 'Seborrheic ...",YES,NaN,NaN,FST4,NaN,NaN,True,True,3.0,2.0,"['erythema gyratum repens', 'seborrheic dermat...",dermatitis,"{'melanoma': 0.31323570013046265, 'psoriasis':..."


In [63]:
existing_files

{-1000600354148496558: '/home/provido/provido/luna/image_data/scin/-3205742176803893704.png',
 -1002039107727665188: '/home/provido/provido/luna/image_data/scin/-4762289084741430925.png',
 -1003358831658393077: '/home/provido/provido/luna/image_data/scin/-4027806997035329030.png',
 -1003826561155964328: '/home/provido/provido/luna/image_data/scin/742075435141960831.png',
 -1003844406100696311: '/home/provido/provido/luna/image_data/scin/5854025080806696361.png',
 -1005079160214352144: '/home/provido/provido/luna/image_data/scin/-3575683440831198879.png',
 -1010778459521153386: '/home/provido/provido/luna/image_data/scin/8479707042238010445.png',
 -1013831220015814987: '/home/provido/provido/luna/image_data/scin/-220649468572111832.png',
 -101827005996397499: '/home/provido/provido/luna/image_data/scin/-217828380359571871.png',
 -1022162013984621110: '/home/provido/provido/luna/image_data/scin/8406712750126834903.png',
 -1031911667791094656: '/home/provido/provido/luna/image_data/scin/8

### LAST THING TO DO: CHOOSE SAME NUM OF SAMPLES PER CLASS
"""
She's going to load them by zip so that samples of same class are zipped together.
What I could do now is preprocess
"""

In [77]:
hcase_ids =cases[cases['related_category'] == 'LOOKS_HEALTHY']['case_id'].tolist()
len(hcase_ids)

290

In [78]:
# hcase_ids is your list of healthy case_ids
mask = df["case_id"].isin(hcase_ids)

df.loc[mask, "assigned_target_label"] = "healthy"

In [ ]:
#before healthy reassignment label
df2['assigned_target_label'].value_counts()

assigned_target_label
unclear                         3878
psoriasis                        481
dermatitis                       348
systemic lupus erythematosus     122
melanoma                          56
scleroderma                       25
vitiligo                          19
Name: count, dtype: int64

In [80]:
# hcase_ids is your list of healthy case_ids
mask = df["case_id"].isin(hcase_ids)

df.loc[mask, "assigned_target_label"] = "healthy"
df['assigned_target_label'].value_counts()

assigned_target_label
unclear                         3735
psoriasis                        472
dermatitis                       328
healthy                          290
systemic lupus erythematosus     111
melanoma                          55
scleroderma                       25
vitiligo                          17
Name: count, dtype: int64

In [94]:
#df_sorted = df.sort_values(by="assigned_target_label", key=lambda s: s.str.lower())
#df_sorted = df_sorted[df_sorted['assigned_target_label'] != 'unclear']
df_sorted['assigned_target_label'].value_counts()


assigned_target_label
psoriasis                       472
dermatitis                      328
healthy                         290
systemic lupus erythematosus    111
melanoma                         55
scleroderma                      25
vitiligo                         17
Name: count, dtype: int64

In [93]:
###NOW SAVING WITH HEALTHY! PERFECT
#df_sorted.to_csv("/home/provido/provido/luna/image_data/scin_meta/final_sorted_samples-images.csv", index = False)
df_sorted = pd.read_csv("/home/provido/provido/luna/image_data/scin_meta/final_sorted_samples-images.csv")
df_sorted

,case_id,dermatologist_gradable_for_skin_condition_2,dermatologist_gradable_for_skin_condition_3,dermatologist_skin_condition_on_label_name,dermatologist_skin_condition_confidence,weighted_skin_condition_label,dermatologist_gradable_for_fitzpatrick_skin_type_1,dermatologist_gradable_for_fitzpatrick_skin_type_2,dermatologist_gradable_for_fitzpatrick_skin_type_3,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,gradable_for_monk_skin_tone_india,gradable_for_monk_skin_tone_us,monk_skin_tone_label_india,monk_skin_tone_label_us,conditions_list,assigned_target_label,assigned_target_scores
0,-2740479848383925335,NaN,NaN,"['Acne', 'Rosacea', 'Comedone']","[5, 5, 4]","{'Acne': 0.41, 'Rosacea': 0.41, 'Comedone': 0.18}",YES,NaN,NaN,FST4,NaN,NaN,True,True,3.0,5.0,"['acne', 'rosacea', 'comedone']",dermatitis,"{'melanoma': 0.29304444789886475, 'psoriasis':..."
1,-5156441036700827606,NaN,NaN,"['resolving dermatitis', ""Kaposi's sarcoma of ...","[3, 5, 5]","{""Kaposi's sarcoma of skin"": 0.41, 'O/E - ecch...",YES,NaN,NaN,FST3,NaN,NaN,True,True,3.0,4.0,"['resolving dermatitis', ""kaposi's sarcoma of ...",dermatitis,"{'melanoma': 0.49743276834487915, 'psoriasis':..."
2,-5197414973622050223,NaN,NaN,"['Tinea', 'Acute and chronic dermatitis']","[2, 2]","{'Tinea': 0.5, 'Acute and chronic dermatitis':...",YES,NaN,NaN,FST3,NaN,NaN,True,True,2.0,3.0,"['tinea', 'acute and chronic dermatitis']",dermatitis,"{'melanoma': 0.37104693055152893, 'psoriasis':..."
3,1713789049192481311,NaN,NaN,"['Porokeratosis', 'Tinea', 'Drug Rash']","[2, 4, 3]","{'Tinea': 0.55, 'Drug Rash': 0.27, 'Porokerato...",YES,NaN,NaN,FST4,NaN,NaN,True,True,4.0,6.0,"['porokeratosis', 'tinea', 'drug rash']",dermatitis,"{'melanoma': 0.24827466905117035, 'psoriasis':..."
4,4093757242794260092,NaN,NaN,"['Acne', 'Eruptive odontogenic cyst']","[3, 2]","{'Acne': 0.67, 'Eruptive odontogenic cyst': 0.33}",YES,NaN,NaN,FST3,NaN,NaN,True,True,2.0,3.0,"['acne', 'eruptive odontogenic cyst']",dermatitis,"{'melanoma': 0.21422062814235687, 'psoriasis':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,8190167325801713856,NaN,NaN,['Melanin pigmentation due to exogenous substa...,[5],{'Melanin pigmentation due to exogenous substa...,NO,NaN,NaN,NaN,NaN,NaN,True,True,2.0,5.0,['melanin pigmentation due to exogenous substa...,vitiligo,"{'melanoma': 0.35537612438201904, 'psoriasis':..."
1294,3299546696242067710,NaN,NaN,"['Scar Condition', 'Lichen planus/lichenoid er...","[2, 2, 2]","{'Scar Condition': 0.33, 'Lichen planus/lichen...",YES,NaN,NaN,FST5,NaN,NaN,True,True,3.0,5.0,"['scar condition', 'lichen planus/lichenoid er...",vitiligo,"{'melanoma': 0.3382207453250885, 'psoriasis': ..."
1295,7375378820280763393,DEFAULT_YES_IMAGE_QUALITY_SUFFICIENT,NO_IMAGE_QUALITY_INSUFFICIENT,"['Photodermatitis', 'Vitiligo']","[2, 1]","{'Photodermatitis': 0.5, 'Vitiligo': 0.5}",YES,YES,YES,FST2,FST4,FST2,True,True,2.0,2.0,"['photodermatitis', 'vitiligo']",vitiligo,"{'melanoma': 0.0, 'psoriasis': 0.0, 'systemic ..."
1296,-6776940077739990895,NaN,NaN,"['Tinea Versicolor', 'Post-Inflammatory hyperp...","[1, 1, 5]",{'Confluent and reticulate papillomatosis': 0....,YES,NaN,NaN,FST3,NaN,NaN,True,True,3.0,5.0,"['tinea versicolor', 'post-inflammatory hyperp...",vitiligo,"{'melanoma': 0.3670094609260559, 'psoriasis': ..."


In [119]:
# Convert dictionary to a Series or DataFrame for merging
file_paths_series = pd.Series(existing_files, name='image_path')

# Merge by matching the dictionary keys with df['case_id']
df_sorted3 = df_sorted2.merge(
    file_paths_series, 
    left_on='case_id', 
    right_index=True, 
    how='left'
)

print(f"Dataframe shape after merge: {df_sorted3.shape}")
print(f"Rows with image_path: {df_sorted3['image_path'].notna().sum()}")
print(f"Rows without image_path: {df_sorted3['image_path'].isna().sum()}")

# Preview
print("\nSample rows:")
print(df_sorted3[['case_id', 'image_path']].head())
df_sorted3 = df_sorted3[~df_sorted3['image_path'].isna()]
df_sorted3

Dataframe shape after merge: (1298, 6)
Rows with image_path: 1269
Rows without image_path: 29

Sample rows:
               case_id                                         image_path
0 -2740479848383925335  /home/provido/provido/luna/image_data/scin/870...
1 -5156441036700827606  /home/provido/provido/luna/image_data/scin/-55...
2 -5197414973622050223  /home/provido/provido/luna/image_data/scin/-38...
3  1713789049192481311  /home/provido/provido/luna/image_data/scin/453...
4  4093757242794260092  /home/provido/provido/luna/image_data/scin/653...


,case_id,dermatologist_skin_condition_on_label_name,conditions_list,assigned_target_label,assigned_target_scores,image_path
0,-2740479848383925335,"['Acne', 'Rosacea', 'Comedone']","['acne', 'rosacea', 'comedone']",dermatitis,"{'melanoma': 0.29304444789886475, 'psoriasis':...",/home/provido/provido/luna/image_data/scin/870...
1,-5156441036700827606,"['resolving dermatitis', ""Kaposi's sarcoma of ...","['resolving dermatitis', ""kaposi's sarcoma of ...",dermatitis,"{'melanoma': 0.49743276834487915, 'psoriasis':...",/home/provido/provido/luna/image_data/scin/-55...
2,-5197414973622050223,"['Tinea', 'Acute and chronic dermatitis']","['tinea', 'acute and chronic dermatitis']",dermatitis,"{'melanoma': 0.37104693055152893, 'psoriasis':...",/home/provido/provido/luna/image_data/scin/-38...
3,1713789049192481311,"['Porokeratosis', 'Tinea', 'Drug Rash']","['porokeratosis', 'tinea', 'drug rash']",dermatitis,"{'melanoma': 0.24827466905117035, 'psoriasis':...",/home/provido/provido/luna/image_data/scin/453...
4,4093757242794260092,"['Acne', 'Eruptive odontogenic cyst']","['acne', 'eruptive odontogenic cyst']",dermatitis,"{'melanoma': 0.21422062814235687, 'psoriasis':...",/home/provido/provido/luna/image_data/scin/653...
...,...,...,...,...,...,...
1293,8190167325801713856,['Melanin pigmentation due to exogenous substa...,['melanin pigmentation due to exogenous substa...,vitiligo,"{'melanoma': 0.35537612438201904, 'psoriasis':...",/home/provido/provido/luna/image_data/scin/-75...
1294,3299546696242067710,"['Scar Condition', 'Lichen planus/lichenoid er...","['scar condition', 'lichen planus/lichenoid er...",vitiligo,"{'melanoma': 0.3382207453250885, 'psoriasis': ...",/home/provido/provido/luna/image_data/scin/-89...
1295,7375378820280763393,"['Photodermatitis', 'Vitiligo']","['photodermatitis', 'vitiligo']",vitiligo,"{'melanoma': 0.0, 'psoriasis': 0.0, 'systemic ...",/home/provido/provido/luna/image_data/scin/-22...
1296,-6776940077739990895,"['Tinea Versicolor', 'Post-Inflammatory hyperp...","['tinea versicolor', 'post-inflammatory hyperp...",vitiligo,"{'melanoma': 0.3670094609260559, 'psoriasis': ...",/home/provido/provido/luna/image_data/scin/857...


In [125]:
df_sorted3.to_csv("/home/provido/provido/luna/image_data/scin_meta/final_sorted_samples-images3.csv", index = False)
df_sorted3['assigned_target_label'].value_counts()

assigned_target_label
psoriasis                       460
dermatitis                      324
healthy                         279
systemic lupus erythematosus    111
melanoma                         55
scleroderma                      24
vitiligo                         16
Name: count, dtype: int64

In [ ]:
df_sorted2 = df_sorted.drop(columns=["dermatologist_gradable_for_skin_condition_2", "dermatologist_gradable_for_skin_condition_3", "dermatologist_skin_condition_confidence",	"weighted_skin_condition_label", "dermatologist_gradable_for_fitzpatrick_skin_type_1", "dermatologist_gradable_for_fitzpatrick_skin_type_2", "dermatologist_gradable_for_fitzpatrick_skin_type_3", "dermatologist_fitzpatrick_skin_type_label_1", "dermatologist_fitzpatrick_skin_type_label_2", "dermatologist_fitzpatrick_skin_type_label_3", "gradable_for_monk_skin_tone_india", "gradable_for_monk_skin_tone_us", "monk_skin_tone_label_india", "monk_skin_tone_label_us"])
df_sorted2.to_csv("/home/provido/provido/luna/image_data/scin_meta/final_sorted_samples-images2.csv", index = False)
df_sorted2

,case_id,dermatologist_skin_condition_on_label_name,conditions_list,assigned_target_label,assigned_target_scores
0,-2740479848383925335,"['Acne', 'Rosacea', 'Comedone']","['acne', 'rosacea', 'comedone']",dermatitis,"{'melanoma': 0.29304444789886475, 'psoriasis':..."
1,-5156441036700827606,"['resolving dermatitis', ""Kaposi's sarcoma of ...","['resolving dermatitis', ""kaposi's sarcoma of ...",dermatitis,"{'melanoma': 0.49743276834487915, 'psoriasis':..."
2,-5197414973622050223,"['Tinea', 'Acute and chronic dermatitis']","['tinea', 'acute and chronic dermatitis']",dermatitis,"{'melanoma': 0.37104693055152893, 'psoriasis':..."
3,1713789049192481311,"['Porokeratosis', 'Tinea', 'Drug Rash']","['porokeratosis', 'tinea', 'drug rash']",dermatitis,"{'melanoma': 0.24827466905117035, 'psoriasis':..."
4,4093757242794260092,"['Acne', 'Eruptive odontogenic cyst']","['acne', 'eruptive odontogenic cyst']",dermatitis,"{'melanoma': 0.21422062814235687, 'psoriasis':..."
...,...,...,...,...,...
1293,8190167325801713856,['Melanin pigmentation due to exogenous substa...,['melanin pigmentation due to exogenous substa...,vitiligo,"{'melanoma': 0.35537612438201904, 'psoriasis':..."
1294,3299546696242067710,"['Scar Condition', 'Lichen planus/lichenoid er...","['scar condition', 'lichen planus/lichenoid er...",vitiligo,"{'melanoma': 0.3382207453250885, 'psoriasis': ..."
1295,7375378820280763393,"['Photodermatitis', 'Vitiligo']","['photodermatitis', 'vitiligo']",vitiligo,"{'melanoma': 0.0, 'psoriasis': 0.0, 'systemic ..."
1296,-6776940077739990895,"['Tinea Versicolor', 'Post-Inflammatory hyperp...","['tinea versicolor', 'post-inflammatory hyperp...",vitiligo,"{'melanoma': 0.3670094609260559, 'psoriasis': ..."
